In [14]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [15]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [16]:
img_folder = r'D:\original'
out_folder = r'D:\ToAWS2'

course_list = ['청도','포항','중문','시흥','거창','속리산','춘천']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
  '시흥':'MGC004',
  '거창':'MGC005',
  '속리산':'MGC006',
  '춘천':'MGC007'
}

folder_type = ['rgb','ndvi','thumb']

In [39]:
course_id = course_ids.get('시흥')
course_id

'MGC004'

In [40]:
target_folders = glob.glob(os.path.join(out_folder,'**/*'))
target_folders = [x for x in target_folders if course_id in x]
target_folders

['D:\\ToAWS2\\MGC004\\20230807',
 'D:\\ToAWS2\\MGC004\\20230815',
 'D:\\ToAWS2\\MGC004\\20230822',
 'D:\\ToAWS2\\MGC004\\20230902',
 'D:\\ToAWS2\\MGC004\\20230905',
 'D:\\ToAWS2\\MGC004\\20230912',
 'D:\\ToAWS2\\MGC004\\20230919']

In [41]:
target_folder = target_folders[1]

In [42]:
target_date = os.path.split(target_folder)[-1]
course_id, target_date

('MGC004', '20230815')

In [43]:
dataJson_ = glob.glob(os.path.join(out_folder,course_id,target_date,'**/photo*.json'), recursive=True)[0]
dataJson_ 

'D:\\ToAWS2\\MGC004\\20230815\\photo.json'

In [44]:
with open(dataJson_, "r") as _json:
  dataJson= json.load(_json)

In [45]:
len(dataJson)

7436

In [46]:
dataJson[0]

{'id': '20230815104825_1267700191111_373687385000_52909000',
 'originalFileJPG': 'NIA_2023_52_잔디\\11_원천데이터\\2023_0815_시흥 아세코밸리_맑음_데이터 1차 분류본_완\\아세코밸리 3H\\2023_0815_시흥 아세코벨리_3H_0.5m\\20230815_1048-1055_잡초,광엽,중대가리풀,애기땅빈대\\DJI_20230815104825_0363_D.JPG',
 'originalFileGrp': ['NIA_2023_52_잔디\\11_원천데이터\\2023_0815_시흥 아세코밸리_맑음_데이터 1차 분류본_완\\아세코밸리 3H\\2023_0815_시흥 아세코벨리_3H_0.5m\\20230815_1048-1055_잡초,광엽,중대가리풀,애기땅빈대\\DJI_20230815104825_0363_MS_G.TIF',
  'NIA_2023_52_잔디\\11_원천데이터\\2023_0815_시흥 아세코밸리_맑음_데이터 1차 분류본_완\\아세코밸리 3H\\2023_0815_시흥 아세코벨리_3H_0.5m\\20230815_1048-1055_잡초,광엽,중대가리풀,애기땅빈대\\DJI_20230815104825_0363_MS_NIR.TIF',
  'NIA_2023_52_잔디\\11_원천데이터\\2023_0815_시흥 아세코밸리_맑음_데이터 1차 분류본_완\\아세코밸리 3H\\2023_0815_시흥 아세코벨리_3H_0.5m\\20230815_1048-1055_잡초,광엽,중대가리풀,애기땅빈대\\DJI_20230815104825_0363_MS_R.TIF',
  'NIA_2023_52_잔디\\11_원천데이터\\2023_0815_시흥 아세코밸리_맑음_데이터 1차 분류본_완\\아세코밸리 3H\\2023_0815_시흥 아세코벨리_3H_0.5m\\20230815_1048-1055_잡초,광엽,중대가리풀,애기땅빈대\\DJI_20230815104825_0363_MS_RE.TIF'],
 'destFolder': '20230

In [47]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [48]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

with table.batch_writer(overwrite_by_pkeys=['metadata', 'id']) as batch:
  for index_, row_ in enumerate(dataJson):
    newrow = {}
    newrow ['id'] = row_['id']
    newrow ['metadata'] = course_id+target_date+row_['info']['area']+row_['info']['desc']
    newrow ['courseid'] = course_id
    newrow ['date'] = target_date
    newrow ['area'] = row_['info']['area']
    newrow ['desc'] = row_['info']['desc']
    newrow ['json'] = json.loads(json.dumps(row_), parse_float=Decimal)
    print(index_, end='\r')
    batch.put_item(
      Item= newrow
    )

In [49]:
courseid = course_id
date = target_date

response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid),  FilterExpression=Attr('date').eq(date))
data =  response['Items']

while 'LastEvaluatedKey' in response:
    response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid), 
     ExclusiveStartKey=response['LastEvaluatedKey'] ,FilterExpression=Attr('date').eq(date) )    
    # response_ = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [50]:
len(data)

7436

In [51]:
courseid, date

('MGC004', '20230815')

In [57]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

courseid = course_id
date = target_date

area = "아세코벨리 1H"

desc = "광엽,화본과,민들래,바랭이,애기땅빈대,왕바랭이,이종잔디,조이시아그래스,사초과,방동사니_0"

response = table.query(KeyConditionExpression=Key('metadata').eq(courseid+date+area+desc))
data =  response['Items']

while 'LastEvaluatedKey' in response:
    response = table.query(KeyConditionExpression=Key('metadata').eq(courseid+date+area+desc),  ExclusiveStartKey=response['LastEvaluatedKey']  )    
    # response_ = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [58]:
len(data)

290